# create tsv files in 'tsv_correct2' directory where we have preprocessed the tsv file (just created in parser.py)

In [ ]:
import string

from shutil import move
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+')
name="aritcle_"
extension2=".tsv"
exclude = string.punctuation
for index in range(len(listUrl_Movies3)):



    file="{}{}{}".format(name,index,extension2)
    with open("tsv/"+file,"r") as tsvfile, open("tsv_correct2/"+file,"w") as outfile:
        tsvreader = csv.reader(tsvfile, delimiter="\t")
        tsvwriter = csv.writer(outfile, delimiter="\t")
        for row in tsvreader:
            for i in range(len(row)):
                #take every words, deleting ountuaction and other symbols
                row[i] = tokenizer.tokenize(row[i])
                #remove duplicate case-insensitive elements
                row[i]= list(map(str.lower, row[i]))
                #row[i] = row[i].translate({ord(c): None for c in string.punctuation})
                
            tsvwriter.writerow(row)
 





# create dataframe that match tfidf bettween every document and every word  

In [ ]:
             

dict2 = {}
count=0
present=False
documents=[]
name='aritcle_'
extension2='.tsv'
with open('tsv/vocobulary.tsv', 'r', newline='') as f_output:
        tsv_vocabulary = list(csv.reader(f_output, delimiter='\t'))
        for index in range(0,10000):
            print(index)
            file="{}{}{}".format(name,index,extension2)
            with open("tsv_correct2/"+file,"r") as tsvfile:
                data_list = list(csv.reader(tsvfile, delimiter="\t"))
                tsvreader = csv.reader(tsvfile, delimiter="\t")
                intro=data_list[1][1]
                intro = ast.literal_eval(intro)
                plot=data_list[1][2]
                plot = ast.literal_eval(plot)
                text=plot+intro
                text=list(map(str.lower, text))
                documents.append(' '.join(text))
                #print(documents)
                # text2= list(set(map(str.lower, text)))
                
                
        vectorizer = TfidfVectorizer(token_pattern=r"(?u)\b\w+\b")
      
        vectors = vectorizer.fit_transform(documents)
        #print(vectors)
        feature_names = vectorizer.get_feature_names()
        dense = vectors.todense()
        denselist = dense.tolist()
        df2 = pd.DataFrame(denselist, columns=feature_names)

# create index.tsv with iditf from preovious dataframe

In [ ]:
#
        
import ast
from itertools import islice
import csv
import sys
csv.field_size_limit(sys.maxsize)
#create index and save it on index.tsv                
dict={}
dict2 = {}
count=0
present=False


print('step 2')
name="aritcle_"
extension2=".tsv"
h=0
for row in tsv_vocabulary:
    h+=1
    dict[row[0]]=row[1]
    print(h)
    dict2[row[1]]=[]
for index in range(0,10000):
    print("numero documento "+ str(index))
    file="{}{}{}".format(name,index,extension2)
    with open("tsv_correct2/"+file,"r") as tsvfile:
        data_list = list(csv.reader(tsvfile, delimiter="\t"))
        tsvreader = csv.reader(tsvfile, delimiter="\t")
        intro=data_list[1][1]
        intro = ast.literal_eval(intro)
        plot=data_list[1][2]
        plot = ast.literal_eval(plot)
        text=plot+intro
        text=list(map(str.lower, text))
    
        text2= list(set(map(str.lower, text)))
        #print(text2)
                        #for evry words in plot adn intro (for every page) we get every word. From every word we get its term_id and put it whit their occurences (document_id) in dict2
        for i in text2:

                            #print("word   "+str(i))
            # print("word "+str(i))
            # print("aaa" + str(df2.iloc[index][i]))
            res=df2.iloc[index][i]
                            #print("res   "+str(res))
            for term in dict:
                if i==term:
                    print("key "+ str(term))
                    print("value "+ str(dict[term]))
                    doc="document_"
                    name2="{}{}".format(doc,index)
                    result=[name2,res]
                    dict2[dict[term]].append(result)
                    break
                else:
                    continue

            

with open('tsv/index.tsv', 'w', newline='') as f_output:
    tsv_index2 = csv.writer(f_output, delimiter='\t')           
    for key, val in dict2.items():
        tsv_index2.writerow([key, val])
   

# functions: to get itidf about query, to get itidf about documnet_id for wirds in query, to calculate coisine similiarity

In [ ]:
#
from sklearn.feature_extraction.text import TfidfVectorizer
def getTfidf_query(query):


    vectorizer = TfidfVectorizer(token_pattern=r"(?u)\b\w+\b")
      
    vectors = vectorizer.fit_transform([query])
        #print(vectors)
    feature_names = vectorizer.get_feature_names()
    dense = vectors.todense()
    denselist = dense.tolist()
    df_query = pd.DataFrame(denselist, columns=feature_names)
    return df_query


#function 
def getTfidf_document(words,document_id):
    dict={}
    
    listWords = words.split()
    listWords=[x.lower() for x in listWords]
    #print(listWords)
    df=pd.DataFrame(columns=listWords)
    tfIdf=[]
    with open('tsv/index.tsv', 'r', newline='') as f_output:
        tsv_index = list(csv.reader(f_output, delimiter='\t'))
        
    for word in listWords:
        listDoc=[]
        #print("word " + word)
        for row in tsv_vocabulary:
            if word.lower()==row[0]:
                term=row[1]
                #print("teerm" + str(term))
                break
        for row in tsv_index:
            if term==row[0]:
                
                listDoc=ast.literal_eval(row[1])
                
                #print(listDoc)
                break
        for index in listDoc:
            
            #print(index)
            if index[0]==document_id:
                #print(index[0])
                
                tfIdf.append(index[1])
                #print(index[1])
                break
        
    df.loc[0]=tfIdf
    df=df.reindex(sorted(df.columns), axis=1)
    return df


from sklearn.metrics.pairwise import cosine_similarity
def coisine(list_query,list_document):
    
    res=(cosine_similarity([list_query,list_document]))
    #print(res)
    return res[0][1]



# get documents that contain query

In [ ]:
import csv
import sys
import ast
#define function that allows us to calculate a list that is an intersection from two list
def intersection(lst1, lst2): 
    return list(set(lst1) & set(lst2)) 
def getDocuments(words):


    #we use dict3 to store term_id and its respective documents_id
    dict3={}
    ##we use dict4 to store evry word and its respective documents_id
    dict4={}
    csv.field_size_limit(sys.maxsize)
    #in listWords we have a list that contains all words about inout query
    listWords = words.split()
    listWords=[x.lower() for x in listWords]

    #with vocabulary.tsv we start to build a dict3 with term_id for every words in wordsList
    with open('HW3 ADM/tsv/vocobulary.tsv', 'r', newline='') as f_output:
        tsv_vocabulary = list(csv.reader(f_output, delimiter='\t'))
        for word in listWords:
            word=word.lower()
            present=False
            for row in tsv_vocabulary:
                if word.lower()==row[0]:
                    dict3[row[1]]=[]
                    present=True
            #case where word is not in vocabulary
            if present==False:
                dict4[word]=[]

        #we continue to match dicumnets_id to every term_id in dict3
        with open('HW3 ADM/tsv/index2.tsv', 'r', newline='') as f_output:
            tsv_index = list(csv.reader(f_output, delimiter='\t'))
            for k in dict3.keys():  
                for row in tsv_index:
                    if row[0]==k:
                        dict3[k]=row[1]
                        continue


        #finally we build dict4 where evry word matches to respective documents_id
        for k in dict3.keys():

            for row in tsv_vocabulary:
                if k==row[1]:
                    dict4[row[0]]=dict3[row[1]]

        document=ast.literal_eval(dict4[listWords[0]])         
        #interection between every list in values dict4. In this way we have documnets_id where all words (in query input) are present
        for value in dict4.values():
            document=intersection(document,ast.literal_eval(value))
        #print(document)
    return document

# function that execute searchEngine2

In [ ]:


def searchEngine2(words):
    #build the dataframe with info for every documents_id
    document=getDocuments(words)
    df_query=getTfidf_query(words)
    df=pd.DataFrame(columns=['title', 'intro', 'url','similarity'])

    for index in range(len(document)):
        #get id of documnets_is
        df_document=getTfidf_document(words,document[index])
        #print(df_document)
        list_query=list(df_query.loc[0])
        #print(list_query)
        list_document=list(df_document.loc[0])
        #print(list_document)
        similiarity=coisine(list_document,list_query)
        #print("simi "+ str(similiarity))
        numberDocument=document[index][9:]
        #get wikipedia url
        url=listUrl_Movies3[int(numberDocument)]
        name="aritcle_"
        extension2=".tsv"
        index=int(numberDocument)
        file="{}{}{}".format(name,index,extension2)
        #get info about title and intro for evert film that corresponds to every documents_id
        with open('tsv/'+file, 'r', newline='') as f_output:
            tsv_index = list(csv.reader(f_output, delimiter='\t'))
            title=tsv_index[1][3]
            intro=tsv_index[1][1]
            film=[title,intro,url,similiarity]
            #put all info for every film in a single row of df dataframe
            df.loc[index] = film
            df=df.sort_values(by=['similarity'],ascending=False)
    return df

# execute searchEngine2

In [ ]:




query='life 2019 horror story'

searchEngine2(query)